# Producing Project Plans
This notebook is for experimenting with having an LLM plan out a coding project.

In [ ]:
# Load environment variables from .env file
from dotenv import load_dotenv
import os
import warnings

from openai import OpenAI

warnings.filterwarnings("ignore")
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print("OPENAI_API_KEY is set correctly.")
else:
    print("OPENAI_API_KEY is not set.")

In [2]:
architecture_policies = [
    {
        "name": "Modularity and Function-Oriented Design",
        "description": "Design the codebase as a collection of small, focused functions, each performing one clear task.",
        "policies": [
            "Each function should have a single responsibility.",
            "Functions should be stateless where possible.",
            "Avoid side effects (e.g., global state modifications).",
            "Write reusable and composable functions.",
        ],
    },
    {
        "name": "CSV as the Core Data Format",
        "description": "Treat CSV files as the primary medium for data input, output, and inter-process communication.",
        "policies": [
            "Input and output data should default to CSV files instead of strings.",
            "Functions should focus on reading from and writing to CSV files.",
            "Encapsulate CSV parsing and writing logic into dedicated utility functions.",
            "Ensure CSV schemas are consistent and well-documented.",
        ],
    },
    {
        "name": "Separation of Concerns",
        "description": "Keep concerns (e.g., data handling, computation, and presentation) isolated.",
        "policies": [
            "Separate data processing, business logic, and I/O operations.",
            "Avoid mixing CSV parsing with computation logic in the same function.",
            "Use interfaces or adapters for external dependencies (e.g., file systems).",
        ],
    },
    {
        "name": "Data Flow Transparency",
        "description": "Ensure clarity in how data moves through the system.",
        "policies": [
            "Functions should explicitly accept inputs and return outputs.",
            "Avoid relying on hidden state or global variables.",
            "Use clear pipelines when chaining multiple data transformations.",
        ],
    },
    {
        "name": "Minimalism (Unix Philosophy)",
        "description": "Build small, sharp tools that can be combined in pipelines.",
        "policies": [
            "Functions should do one thing well.",
            "Allow for chaining outputs to other scripts/tools.",
            "Favor simplicity over cleverness in implementation.",
            "Document command-line or API interfaces for integration.",
        ],
    },
]
policy_list = list(
    set(policy for item in architecture_policies for policy in item.get("policies", []))
)
policies = "\n\t- ".join(policy_list)

In [3]:
project_plan_template = """
#################################
#       PROJECT PLAN  TEMPLATE  #
#################################

## 1. Identify the Architectural Style
Choose one:
- Layered Architecture: Presentation, Business Logic, Data Access, Database
- Microservices Architecture: Independent services for specific capabilities
- Modular Monolith: Independent modules, shared deployment
- Event-Driven Architecture: Event producers, consumers, queues

## 2. Component Interface Definition
Define clear APIs or interfaces:
- Accepted Data: [Specify data input]
- Returned Data: [Specify data output]
- Communication Method: (e.g., REST, gRPC, Message Queue)

## 3. Domain-Driven Design (DDD)
Define bounded contexts:
- User Context: [Responsibilities]
- Order Context: [Responsibilities]
- Payment Context: [Responsibilities]

## 4. Dependency Management
- Dependencies Between Components: [List dependencies]
- Cyclic Dependency Avoidance: [Strategies]
- Dependency Injection: [Yes/No, describe]

## 5. Data Flow Between Sub-Components
Describe data flow:
- Sequence Diagram: [Describe interactions]
- Data Flow Diagram: [Describe flow]
- Component Diagram: [Describe dependencies]

## 6. Component Documentation
Document each component:
- **Name:** [Unique Identifier]
- **Purpose:** [Description of purpose]
- **Inputs:** [Expected inputs]
- **Outputs:** [Expected outputs]
- **Dependencies:** [List dependencies]

## 7. Visualization with Diagrams
Provide or reference diagrams:
- Component Diagram: [High-level overview]
- Deployment Diagram: [Physical deployment setup]
- Sequence Diagram: [Interaction flow]

##############################
#       END OF TEMPLATE      #
##############################
"""

In [4]:
project_ideas = [
    "Summarizer for long text documents, generating concise summaries.",
    "Email analyzer to find key participants, topics, and action items.",
    "Task prioritizer using CSV input for deadlines, importance, and dependencies.",
    "Data cleaner for CSVs, detecting missing values and standardizing formats.",
    "Web scraper to extract key info from URLs and save it into a CSV file.",
]

In [5]:
project_plan_prompt_preamble = "You are a software engineer tasked with designing a new project. Your goal is to create a project plan that follows best practices in software architecture and coding.\n"
project_plan_prompt = f"{project_plan_prompt_preamble}\nYour project should adhere to the following policies::\n {policies}\n\nSolve this problem:\n {project_ideas[0]}\n\nFollow this template:\n {project_plan_template}\n"

In [ ]:
print(project_plan_prompt)

In [ ]:

client = OpenAI(api_key=openai_api_key)

response = client.chat.completions.create(
    model="o1-mini", messages=[{"role": "user", "content": project_plan_prompt}]
)
plan = response.choices[0].message.content

In [ ]:
print(plan)